In [1]:
from __future__ import print_function
import requests
import re
session = requests.session()

# movie_lists = ['http://www.moviesubtitles.org/movies.html'] + \
#               ['http://www.moviesubtitles.org/movies_%s.html' % chr(c) for c in xrange(66, 91)] + \
#               ['http://www.moviesubtitles.org/movies_#.html']

BASE_URL = 'http://www.tvsubtitles.net/'
TV_LIST = 'http://www.tvsubtitles.net/tvshows.html'
response = session.get(TV_LIST)
response.raise_for_status()

tvlist_content = response.content
shows = re.findall(r'<a href="(tvshow-.*?\.html)"', tvlist_content)
print('number of tv series:', len(shows))

number of tv series: 1777


In [2]:
from collections import defaultdict

subtitle_urls = defaultdict(list)
errors = {}
for i, link in enumerate(shows):
    try:
        if i % 10 == 9:
            session = requests.session()
        show_url = BASE_URL + link
        response = session.get(show_url)
        response.raise_for_status()
        result = re.findall('<td><nobr><a href="(subtitle[\d\-]*?-en\.html)"', response.content)
        
        season_links = []
        if result:
            subtitle_urls[link].append(result[-1])

            # Find all seasons.
            season_links = re.findall(r'<a href="(tvshow[\d\-]*?\.html)"><b>Season \d+</b></a>', response.content)
            for sl in season_links:
                season_url = BASE_URL + sl
                response = session.get(season_url)
                response.raise_for_status()
                result = re.findall('<td><nobr><a href="(subtitle[\d\-]*?-en\.html)"', response.content)
                if result:
                    subtitle_urls[link].append(result[-1])

        print(i, link, '\t(%s) for (%s)seasons.' % (len(subtitle_urls[link]), len(season_links)+1))
    except Exception as e:
        errors[link] = e
        print(i, link, '\t'+e.message)

0 tvshow-634-10.html 	(0) for (1)seasons.
1 tvshow-486-1.html 	(1) for (1)seasons.
2 tvshow-1769-1.html 	(0) for (1)seasons.
3 tvshow-700-1.html 	(1) for (1)seasons.
4 tvshow-1110-1.html 	(1) for (1)seasons.
5 tvshow-1938-1.html 	(1) for (1)seasons.
6 tvshow-1655-1.html 	(1) for (1)seasons.
7 tvshow-875-3.html 	(0) for (1)seasons.
8 tvshow-1226-1.html 	(1) for (1)seasons.
9 tvshow-739-2.html 	(0) for (1)seasons.
10 tvshow-899-2.html 	(0) for (1)seasons.
11 tvshow-985-5.html 	(5) for (5)seasons.
12 tvshow-38-9.html 	(9) for (9)seasons.
13 tvshow-451-1.html 	(1) for (1)seasons.
14 tvshow-46-7.html 	(7) for (7)seasons.
15 tvshow-943-1.html 	(0) for (1)seasons.
16 tvshow-653-6.html 	(0) for (1)seasons.
17 tvshow-1171-2.html 	(0) for (1)seasons.
18 tvshow-271-1.html 	(1) for (1)seasons.
19 tvshow-1205-1.html 	(1) for (1)seasons.
20 tvshow-419-11.html 	(0) for (1)seasons.
21 tvshow-642-3.html 	(0) for (1)seasons.
22 tvshow-244-5.html 	(5) for (5)seasons.
23 tvshow-361-4.html 	(4) for (4)seas

KeyboardInterrupt: 

In [3]:
import json
print(json.dumps(subtitle_urls, indent=2))

{
  "tvshow-1567-1.html": [
    "subtitle-1567-1-en.html"
  ], 
  "tvshow-739-2.html": [], 
  "tvshow-616-4.html": [], 
  "tvshow-1075-3.html": [], 
  "tvshow-993-1.html": [
    "subtitle-993-1-en.html"
  ], 
  "tvshow-451-1.html": [
    "subtitle-451-1-en.html"
  ], 
  "tvshow-361-4.html": [
    "subtitle-361-4-en.html", 
    "subtitle-361-3-en.html", 
    "subtitle-361-2-en.html", 
    "subtitle-361-1-en.html"
  ], 
  "tvshow-38-9.html": [
    "subtitle-38-9-en.html", 
    "subtitle-38-8-en.html", 
    "subtitle-38-7-en.html", 
    "subtitle-38-6-en.html", 
    "subtitle-38-5-en.html", 
    "subtitle-38-4-en.html", 
    "subtitle-38-3-en.html", 
    "subtitle-38-2-en.html", 
    "subtitle-38-1-en.html"
  ], 
  "tvshow-1205-1.html": [
    "subtitle-1205-1-en.html"
  ], 
  "tvshow-537-8.html": [], 
  "tvshow-1655-1.html": [
    "subtitle-1655-1-en.html"
  ], 
  "tvshow-899-2.html": [], 
  "tvshow-1110-1.html": [
    "subtitle-1110-1-en.html"
  ], 
  "tvshow-46-7.html": [
    "subtitle-

In [4]:
with open('subtitle_urls.json', 'w') as f:
    f.write(json.dumps(subtitle_urls, indent=2))

In [5]:
import logging
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='tvsub.log', mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.INFO)
logging.getLogger("requests").setLevel(logging.WARNING)


In [6]:
for i, (key, urls) in enumerate(subtitle_urls.iteritems()):
    if i<1235:
        continue
    for url in urls:
        try:
            download_url = BASE_URL + url.replace('subtitle', 'download')
            response = session.get(download_url)

            response.raise_for_status()
            assert 'application/zip' == response.headers['Content-Type']

            fname = url.replace('subtitle-', '').replace('.html', '.zip')
            with open('/Volumes/HDD/CS4300/tvsubtitles/' + fname, 'w') as f:
                for block in response.iter_content(16384):
                    f.write(block)
        except Exception as e:
            logger.exception(e)
    logger.info('%d %s' % (i, key))

In [7]:
response.headers['Content-Type']

'text/html; charset=utf-8'